In [1]:


# Cоединим библиотеку с базой данных (зайдем в Posgresql, после введения пользовательского пароля выберем  "Servers",затем кликаем правой кнопкой мыши, в сплывающем окне выбираем "Properties", переходим в раздел "Connection".
# Увидим отображенные данные

import psycopg2 
#  - драйвер для postgresql
import getpass  
# можно скрыть пароль, но не обязательно  
# в строке password необходимо указать пароль от базы данных 

conn=psycopg2.connect(user='postgres',
                      database='postgres',
                      host='localhost',
                      port='5433',
                      password=getpass.getpass('Insert password: ')
                        )
print(conn.get_dsn_parameters())




Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [ ]:
import pandas as pd


# Увидим таблицу с рейтингом и его типом (используется школа агенства Эксперт РА)
# Строка для указания промежутка времни 
\"DATE\" >=  AND \"DATE\" <=   


query = """SELECT rating_task1.\"GRADE\", rating_task1.\"RAT_TYPE\"
        FROM rating_task1 
        WHERE "AGENCY_ID" = 'EXP'
        AND \"DATE\" >= '2014/01/01' AND \"DATE\" <= '2015/01/01' 
        AND \"CHANGE\" != 'снят' AND \"CHANGE\" != 'приостановлен'"""

ratings=pd.read_sql_query(query, conn)
ratings


In [ ]:
# Выведем БД в таблицу с помощью запроса 


query1 = """SELECT * 
FROM public."scale_EXP_task1" 
WHERE \"GRADE\" != 'Снят' AND \"GRADE\" != 'Приостановлен'"""

rat=pd.read_sql_query(query1, conn)
rat


In [ ]:
# Переведем 'GRADE_ID' в числовой формат

rat['GRADE_ID'] = rat['GRADE_ID'].astype(int)

# Объединим таблицы  

x = pd.merge(rat, ratings, how='inner', on=['GRADE'])
x

# С помощью функции pivot создадим сводную таюлицу 
# Выведем количество компаний допустивших и недопустивших дефолт на заданном промежутке времени, сгруппированнуем по оценке

ratings1 = x[['GRADE_ID', 'RAT_TYPE']]
ratings1['def'] = ratings1['RAT_TYPE']
ratings1['def'] = ratings1['def'].replace(to_replace = {'Cr&D': 'default', 'FS&R': 'non-default', 'R&QS': 'non-default'})
ratings1 = ratings1.pivot_table(index = 'GRADE_ID', columns = 'def', aggfunc = 'count')
ratings1 = ratings1.replace(to_replace = {float('nan'): 0})
ratings1


In [ ]:
import matplotlib.pyplot as plt

# Создадим графика (bar chaert),зададим цвета для разных групп
# допустили дефолт - красный, не допустили дефолт - зеленый

plt.bar(ratings1.index.values, ratings1['RAT_TYPE']['default'].values, label = 'Non-default', color = 'g')
plt.bar(ratings1.index.values, ratings1['RAT_TYPE']['non-default'].values,  label = 'Default', color = 'r')

# Легенда 

plt.legend()

data1 = '2014/01/01'
data2 = '2015/01/01'

# Заголовок Таблицы

plt.title('Распределение допустивших и недопустивших дефолт заемщиков по рейтинговым категориям за период с  ' + str(data1) + ' до ' + str(data2))

plt.show()

plt.savefig('Распределение допустивших и недопустивших дефолт заемщиков по рейтинговым категориям.png')


In [ ]:
# Добавим  функции распределения по рейтинговым категориям, по группам допустившим и не допустившим дефолт

import pandas as pd


ratings2 = pd.DataFrame(index = ratings1.index)


ratings2['non-default'] = ratings1['RAT_TYPE'][0]
ratings2['default'] = ratings1['RAT_TYPE'][1]
ratings2['default_cumsum'] = ratings2['default'].cumsum()/ratings2['default'].sum()
ratings2['non-default_cumsum'] = ratings2['non-default'].cumsum()/ratings2['non-default'].sum()
ratings3 = ratings2[['default_cumsum', 'non-default_cumsum']]
ratings3.plot(kind = 'bar', figsize = [10,10], fontsize = 10)
plt.show()
plt.savefig('Распределение недефолтеров и дефолтеров.png')


In [ ]:
from __future__ import print_function
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML, Document
import os

env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("myreport.html") 
template_vars = {"title": "Отчет составила Пантелеева Ирина",
                "INTRO": "Данный отчет содержит информации о распределении дефолтеров и недефолтеров по рейтинговым категориям", 
                 "TABLE_DESC": "Таблица. В таблице представлено количество компаний, допустивших и недопустивших дефолт, по рейтинговым категориям",
                 "TABLE": ratings1.to_html(), 
                 "IMAGE_DESC": "Рисунок. На графике изображено распределение компаний, допустивших и недопустивших дефолт, по рейтинговым категориям", 
                 "IMAGE": os.path.realpath('Распределение допустивших и недопустивших дефолт заемщиков по рейтинговым категориям.png')
                }


# os.path.realpath - путь к файлу в папке на mac os



# Собираем и выводим финальные данные
html_out = template.render(template_vars, presentational_hints=True)
HTML(string=html_out).write_pdf('my_report.pdf', presentational_hints=True)